In [1]:
import os, code, sys, time, argparse

import numpy as np
from sklearn.neighbors import kneighbors_graph
import tensorflow as tf

import utils
import nn
from utils import REDSHIFTS, PARAMS_SEED, LEARNING_RATE, RS_TAGS

In [2]:
n_P = 32
N = n_P**3
M = 14
k = 3
b = 4
redshifts = [15, 19]

In [3]:
# Load data
X = utils.load_zuni_npy_data(redshifts=redshifts, norm_coo=True)[...,:-1]
Y = X[1] # (1000, N, 6)
X = X[0]

LD: X_0.2280_.npy
LD: X_0.0000_.npy


In [4]:
_x_batch = utils.next_minibatch(X[None,...], b, data_aug=False) # shape (2, b, N, 6)

# split data
x_in    = _x_batch[0] # (b, N, 6)
#x_truth = _x_batch[1] # (b, N, 6)

def graph_get_func(h_in, offset=False, inc_self=False): # for tf.py_func
    return nn.get_kneighbor_list(h_in[...,:3], M, offset_idx=offset, inc_self=inc_self)

# get adj_list
#adj_list = graph_get_func(x_in) # (b, N, M)
#ilist    = nn.alist_to_indexlist(adj_list)
adj_csrs = graph_get_func(x_in)
adj_csrs_offset = graph_get_func(x_in, offset=True)

# get edges (relative dists) and nodes (velocities)
#x_in_edges = nn.get_input_edge_features(x_in, adj_list) # (b, N, M, 3)
#x_in_nodes = nn.get_input_node_features(x_in) # (b, N, 3)

In [5]:
edges = nn.get_input_edge_features_batch(x_in, adj_csrs, M)
edges_off = nn.get_input_edge_features_batch_offset(x_in, adj_csrs_offset, M)
np.all(edges == edges_off)

True

In [6]:
sess = tf.InteractiveSession()

In [7]:
def usm(x, idx, num_seg):
    return tf.unsorted_segment_mean(x, idx, num_seg).eval()

In [8]:
c = 5
P = 18
Q = 4
#h = np.random.randint(0,30,(c, P*Q, 3)) # N: 6, M:4
h = np.random.rand(c, P*Q, 3)
#idx = np.random.randint(0,4,(4,24)) # (b, N*M)
idx = np.zeros((c, P, Q)).astype(np.int32)
offset_idx = np.zeros((c, P, Q)).astype(np.int32)
valid_idx = np.arange(P)
for i in range(c):
    for j in range(P):
        cur_idx = np.random.choice(valid_idx, (Q,), replace=False)
        idx[i, j] = cur_idx
        offset_idx[i,j] = cur_idx + P*i
idx.shape

(5, 18, 4)

In [9]:
idx_in = idx.reshape(c, P*Q)

In [10]:
per_batch = P * np.expand_dims(np.arange(c), axis=1) # (c,1): [[0], [6], [12]]
idx_per_batch = idx_in + per_batch #
idx_per_batch.shape

(5, 72)

In [11]:
usm(h, idx_in, c*P)

array([[0.56220196, 0.56475895, 0.56884619],
       [0.60830483, 0.45381395, 0.53092566],
       [0.45686878, 0.46888358, 0.41427999],
       [0.58757891, 0.63686427, 0.52139013],
       [0.54094249, 0.50865011, 0.40339861],
       [0.50349105, 0.53142113, 0.53177324],
       [0.59335399, 0.56382008, 0.40938047],
       [0.53159837, 0.5987882 , 0.50662026],
       [0.54883862, 0.50371463, 0.49570818],
       [0.56802666, 0.42547078, 0.45411204],
       [0.55843458, 0.50238106, 0.58704892],
       [0.65211189, 0.57287591, 0.62403884],
       [0.36603159, 0.57554931, 0.5208163 ],
       [0.54004965, 0.46792567, 0.60603872],
       [0.39448266, 0.37647996, 0.52313047],
       [0.60184176, 0.53146847, 0.55786462],
       [0.49658232, 0.41127507, 0.43898134],
       [0.41460781, 0.41220429, 0.45711023],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.

In [12]:
usm(h, idx_per_batch, c*P)

array([[0.59839493, 0.70479465, 0.29547506],
       [0.18138043, 0.37602288, 0.35452454],
       [0.25189665, 0.5776658 , 0.32032362],
       [0.76113761, 0.7112228 , 0.42515035],
       [0.5213481 , 0.49972901, 0.32652361],
       [0.5352074 , 0.61006791, 0.55605753],
       [0.78461775, 0.6472125 , 0.48045318],
       [0.45779136, 0.46462203, 0.52873717],
       [0.4563501 , 0.63918042, 0.68291369],
       [0.25081179, 0.49998045, 0.59403923],
       [0.62193076, 0.48681645, 0.69941903],
       [0.44708912, 0.22383969, 0.4423136 ],
       [0.39838417, 0.84938233, 0.62227569],
       [0.49609876, 0.59384277, 0.54787607],
       [0.55656048, 0.30052474, 0.74795301],
       [0.41426686, 0.46477691, 0.23121312],
       [0.56923067, 0.3520924 , 0.36071774],
       [0.35685033, 0.23319404, 0.44448198],
       [0.24662306, 0.43542411, 0.75349626],
       [0.61960428, 0.24797009, 0.95360338],
       [0.57421602, 0.35628421, 0.36901222],
       [0.65146507, 0.60789415, 0.6321274 ],
       [0.

In [15]:
offset_idx_r = np.reshape(offset_idx, (c, P*Q))
off_mu = usm(h, offset_idx_r, c*P)
off_mu

array([[0.59839493, 0.70479465, 0.29547506],
       [0.18138043, 0.37602288, 0.35452454],
       [0.25189665, 0.5776658 , 0.32032362],
       [0.76113761, 0.7112228 , 0.42515035],
       [0.5213481 , 0.49972901, 0.32652361],
       [0.5352074 , 0.61006791, 0.55605753],
       [0.78461775, 0.6472125 , 0.48045318],
       [0.45779136, 0.46462203, 0.52873717],
       [0.4563501 , 0.63918042, 0.68291369],
       [0.25081179, 0.49998045, 0.59403923],
       [0.62193076, 0.48681645, 0.69941903],
       [0.44708912, 0.22383969, 0.4423136 ],
       [0.39838417, 0.84938233, 0.62227569],
       [0.49609876, 0.59384277, 0.54787607],
       [0.55656048, 0.30052474, 0.74795301],
       [0.41426686, 0.46477691, 0.23121312],
       [0.56923067, 0.3520924 , 0.36071774],
       [0.35685033, 0.23319404, 0.44448198],
       [0.24662306, 0.43542411, 0.75349626],
       [0.61960428, 0.24797009, 0.95360338],
       [0.57421602, 0.35628421, 0.36901222],
       [0.65146507, 0.60789415, 0.6321274 ],
       [0.

In [16]:
h_r = h.reshape(-1, 3)
off_r = offset_idx.reshape(-1)
h_sel = h_r[off_r]
h_sel = h_sel.reshape(c, P, Q, 3)

In [17]:
sel_mu = np.mean(h_sel, axis=2).reshape(-1, 3)
sel_mu

array([[0.80850077, 0.69084944, 0.61418011],
       [0.4131754 , 0.62480815, 0.59009931],
       [0.62246379, 0.53424258, 0.62115731],
       [0.81673578, 0.55963745, 0.59066753],
       [0.79367569, 0.75377433, 0.5856494 ],
       [0.61642751, 0.68668243, 0.49987048],
       [0.74317221, 0.67194425, 0.50414005],
       [0.37147278, 0.70282442, 0.62490587],
       [0.78110711, 0.75566705, 0.64089102],
       [0.63657585, 0.42995664, 0.52588818],
       [0.81523586, 0.54654579, 0.65603176],
       [0.5064458 , 0.6493537 , 0.69753598],
       [0.53027357, 0.77329008, 0.65430319],
       [0.54268227, 0.4652261 , 0.447331  ],
       [0.78915573, 0.68223819, 0.50820342],
       [0.63944084, 0.51373207, 0.47092114],
       [0.4598804 , 0.63833308, 0.73383997],
       [0.39589673, 0.65600099, 0.48660606],
       [0.58155734, 0.68839683, 0.44626668],
       [0.23311832, 0.76466091, 0.37145399],
       [0.42393379, 0.37868837, 0.5087847 ],
       [0.74913222, 0.36379998, 0.35061853],
       [0.

In [117]:
off_mu

array([[19.25      , 15.        , 24.25      ],
       [12.66666667, 19.66666667, 18.        ],
       [19.4       , 13.6       , 24.8       ],
       [17.66666667,  5.33333333, 13.33333333],
       [19.25      , 16.25      , 22.5       ],
       [14.        , 14.5       , 20.        ],
       [ 8.25      ,  9.        , 14.        ],
       [ 3.5       , 20.5       , 27.5       ],
       [14.75      , 18.5       , 11.75      ],
       [13.25      ,  7.        , 20.75      ],
       [15.33333333,  9.66666667, 11.83333333],
       [19.25      ,  7.75      , 13.25      ],
       [12.66666667, 12.33333333, 12.33333333],
       [18.66666667,  4.        , 10.66666667],
       [14.5       , 16.5       ,  6.        ],
       [19.75      , 11.        ,  6.        ],
       [15.6       , 14.4       , 13.6       ],
       [17.75      , 14.75      ,  5.5       ]])

In [ ]:
h = np.random.randint(0,30,(c, P*Q, 3)) # N: 6, M:4